In [27]:
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from icecream import ic

In [33]:
class FabricPhysicalConstants:
    rho= 2300 #kg/m^3 
    cp= 750 # J/(kg-K) 
    k=0.8 # W/(m-K)
    h = 4 # W/(m-K)
    T0= 293.15 # K 

    alpha = k/(cp*rho) # thermal diffusivity

In [4]:
# in example => th = 0.10 m, dt = 0.010 m 
thickness = 0.10 
dx = 0.010

In [36]:
x_vals = np.arange(start=0, stop=thickness, step=dx)
M = len(x_vals)
M

10

In [37]:
dt = 15 # seconds
period = dt * 4 * 60/20 # 1 hour/ 
times = np.arange(start=0, stop=period , step=dt)
N = len(times)
# TODO resolve difference between this time formulation and that of indoor air => ALSO need to do check of tau 

In [22]:
# initialize 
Txt = np.zeros((len(times), len(x_vals)))
Txt[0,:] = f.T0
Txt.shape

(12, 10)

In [35]:
f = FabricPhysicalConstants()
tau = f.alpha * dt / (dx**2)
Ta =  np.zeros(len(x_vals))
Tb = np.zeros(len(x_vals))

# beta => for convection 
beta = f.h * dx / f.k

tau, beta

# Ta = Tb = x => calling these the same pobject => pointing to the same place 

(0.06956521739130435, 0.049999999999999996)

In [42]:
def calc_boundary(i):
    # TODO beta should be different based on side of wall 
    T0 = (1 -2*tau - 2*tau*beta)*Txt[i,0] + 2*tau*Txt[i, 1] + 2*tau*beta

    TM = (1 -2*tau - 2*tau*beta)*Txt[i,M-1] + 2*tau*Txt[i, M-2] + 2*tau*beta

    return T0, TM


In [58]:
def calc_transient_wall_conduction(f:FabricPhysicalConstants):
    # x vals init 
    x_vals = np.arange(start=0, stop=thickness, step=dx)
    M = len(x_vals)

    # time vals init 
    dt = 15 # seconds
    period = dt * 4 * 60/20 # 1 hour/ 
    times = np.arange(start=0, stop=period , step=dt)
    N = len(times)
    # TODO resolve difference between this time formulation and that of indoor air => ALSO need to do check of tau based on Cengel 

    # initialize big array 
    # # TODO should be called Ttx because t by x 
    Txt = np.zeros((N, M))
    Txt[0,:] = f.T0


    # initialize intermediate arrays for interior node calculation => # TODO put in own function 
    Ta =  np.zeros(M)
    Tb = np.zeros(M)
    ic(len(Ta), len(Tb), Txt.shape)

    # beta => for convection 
    beta = f.h * dx / f.k
    tau = f.alpha * dt / (dx**2)

    for i, _ in enumerate(times):
        row = Txt[i,:] # row at a given timestep
        for m, _ in enumerate(Txt[i,:]): # iterate over the interior nodes
            m = m + 1
            if m < len(x_vals) - 1:
                Ta[m] = row[m - 1] + row[m + 1]
                Tb[m] = row[m]

            if i < len(times) - 1:
                Txt[i+1, :] = Ta*tau + Tb*(1 - 2*tau)
                # update boundary conditions 
                T0, TM = calc_boundary(i)
                Txt[i+1, 0] = T0
                Txt[i+1, M-1] = TM

    return Txt

In [59]:
Txt = calc_transient_wall_conduction(FabricPhysicalConstants())

ic| len(Ta): 10, len(Tb): 10, Txt.shape: (12, 10)


In [60]:
fig = go.Figure()

for t, x_at_time in enumerate(Txt):
    fig.add_trace(go.Scatter(x=x_vals, y=x_at_time, mode='lines+markers', name=t))
fig.update_layout(xaxis_title='Distance Along Wall (X)',
                  yaxis_title='Temperature (ºC)',
                  title='Heat Conduction in Wall')

# TODO => temp at wall edges should be outdoor and indorr temps respectively... 

In [45]:
for i, _ in enumerate(times):
    row = Txt[i,:] # row at a given timestep
    for m, _ in enumerate(Txt[i,:]): # iterate over the interior nodes
        m = m + 1
        if m < len(x_vals) - 1:
            Ta[m] = row[m - 1] + row[m + 1]
            Tb[m] = row[m]

    if i < len(times) - 1:
        Txt[i+1, :] = Ta*tau + Tb*(1 - 2*tau)
        # update boundary conditions 
        T0, TM = calc_boundary(i)
        Txt[i+1, 0] = T0
        Txt[i+1, M-1] = TM

In [46]:
Txt

array([[  0.        , 239.98930632, 289.2888143 , 293.05131155,
        293.15      , 293.15      , 293.05131155, 289.2888143 ,
        239.98930632,   0.        ],
       [ 33.39677305, 226.72392904, 286.12102225, 292.79643789,
        293.14313472, 293.14313472, 292.79643789, 286.12102225,
        226.72392904,  33.39677305],
       [ 60.06909547, 217.40705511, 282.45342729, 292.35617919,
        293.11901668, 293.11901668, 292.35617919, 282.45342729,
        217.40705511,  60.06909547],
       [ 81.54867875, 210.98677077, 278.61734936, 291.72035906,
        293.06594972, 293.06594972, 291.72035906, 278.61734936,
        210.98677077,  81.54867875],
       [ 98.99711814, 206.68711766, 274.82412717, 290.90245165,
        292.97234341, 292.97234341, 290.90245165, 274.82412717,
        206.68711766,  98.99711814],
       [113.29835551, 203.93560531, 271.20265343, 289.92795198,
        292.82835094, 292.82835094, 289.92795198, 271.20265343,
        203.93560531, 113.29835551],
       [12

In [15]:
Txt[1]

array([  0.        , 272.75695652, 293.15      , 293.15      ,
       293.15      , 293.15      , 293.15      , 293.15      ,
       272.75695652,   0.        ])

In [16]:
Txt[2]

array([  0.        , 255.20120605, 291.7313535 , 293.15      ,
       293.15      , 293.15      , 293.15      , 291.7313535 ,
       255.20120605,   0.        ])

In [17]:
Txt[3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
row_num = 0
row = Txt[row_num,:]
for ix, _ in enumerate(Txt[row_num,:]):
    ix = ix + 1
    if ix < len(x_vals) - 1:
        Ta[ix] = row[ix - 1] + row[ix + 1]
        Tb[ix] = row[ix]

print(Ta, Tb)

Txt[1, :] = Ta*tau + Tb*(1 - 2*tau)

Txt[1, :]
    

[  0.  586.3 586.3 586.3 586.3 586.3 586.3 586.3 586.3   0. ] [  0.   293.15 293.15 293.15 293.15 293.15 293.15 293.15 293.15   0.  ]


array([  0.  , 293.15, 293.15, 293.15, 293.15, 293.15, 293.15, 293.15,
       293.15,   0.  ])

In [11]:
row_num = 1
row = Txt[row_num,:]
for ix, _ in enumerate(Txt[row_num,:]):
    ix = ix + 1
    if ix < len(x_vals) - 1:
        Ta[ix] = row[ix - 1] + row[ix + 1]
        Tb[ix] = row[ix]

print(Ta, Tb)

Txt[row_num+1, :] = Ta*tau + Tb*(1 - 2*tau)

Txt[row_num+1, :]

[  0.   293.15 586.3  586.3  586.3  586.3  586.3  586.3  293.15   0.  ] [  0.   293.15 293.15 293.15 293.15 293.15 293.15 293.15 293.15   0.  ]


array([  0.        , 272.75695652, 293.15      , 293.15      ,
       293.15      , 293.15      , 293.15      , 293.15      ,
       272.75695652,   0.        ])